# Лабораторная работа №2
## по дисциплине *Моделирование инженерных задач*
Работу выполнил: студент гр. **М1О-414Бки-19** *Эбиоле Мандомбо Белтран*

**Вариант 4**

Задачей лабораторной работы является построение моделей для классификации белых или красных вин

### Красное вино

In [10]:
# Повторение первых шагов из 1 лабораторной работы
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

dataset = pd.read_csv('datasets/winequality-red.csv', sep=';')

quality_bins = (2,6.5,8)    #Больше 6.5 - отличное вино, иначе - обычное вино
qualities = ['normal','elite']
categories = pd.cut(dataset['quality'], quality_bins, labels = qualities)
dataset['quality'] = categories

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder

preprocessing = ColumnTransformer(
    [('encoder', OrdinalEncoder(), ['quality'])],
    remainder=(MinMaxScaler()),    #Остальные столбцы масштабируем
    verbose_feature_names_out=False)    #Не добавляем префикс к названиям столбцов 
preprocessing.set_output(transform = 'pandas')

dataset = preprocessing.fit_transform(dataset)
dataset.head()

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,1.0,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846
1,1.0,0.283186,0.520548,0.00,0.116438,0.143573,0.338028,0.215548,0.494126,0.362205,0.209581,0.215385
2,1.0,0.283186,0.438356,0.04,0.095890,0.133556,0.197183,0.169611,0.508811,0.409449,0.191617,0.215385
3,1.0,0.584071,0.109589,0.56,0.068493,0.105175,0.225352,0.190813,0.582232,0.330709,0.149701,0.215385
4,1.0,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846


После обработки данные находятся в промежутке от 0 до 1, что упрощает работу с ними. Помимо этого, вместо строкого представления качество теперь представлено как числа 0 и 1, что также упрощает работу.

Перед тем, как подавать данные на вход моделям для обучения, данные делятся на выборки:

In [12]:
from sklearn.model_selection import StratifiedShuffleSplit

labels = dataset['quality']
data = dataset.copy().drop('quality', axis=1)

stratif_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=414)
for train, test in stratif_split.split(data, labels):
    pass

X_test = data.iloc[test]
y_test = labels.iloc[test]

X_train = data.iloc[train]
y_train = labels.iloc[train]

Чтобы не повторять все действия для каждой модели, действия объединим в одну функцию:

In [13]:
import pickle
from scipy.stats import randint, uniform
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

def learn_save_score(clf, params, X_train, y_train, X_test, y_test, path, name):
    result_score = pd.DataFrame({'scores' : ['Accuracy', 'Recall', 'Precision', 'ROC AUC curve']})

    model = GridSearchCV(clf, params)
    model.fit(X_train, y_train)

    with open(f"saved_models/{path}/{name}.pkl", "wb") as file:
        pickle.dump(model, file, protocol=3)
    with open(f"saved_models/{path}/{name}.txt", "w") as file:
        file.write(str(model.best_params_))

    y_pred = model.predict(X_test)  # Результаты показанные моделью
    y_true = np.array(y_test)   # Истинные результаты из тестовой выборки

    result_score[name] = [  accuracy_score(y_true, y_pred), recall_score(y_true, y_pred),
                            precision_score(y_true, y_pred), roc_auc_score(y_true, y_pred)]

    print(f'Confusion matrix:\n {confusion_matrix(y_true, y_pred)}\n')
    return result_score

#### Логистическая регрессия

In [14]:
from sklearn.linear_model import LogisticRegression
log_regression = LogisticRegression(max_iter=500)

params = [
    {'C': [1, 25], 'solver': ['lbfgs'], 'penalty': ['l2']},
    {'C': [1, 25], 'solver': ['liblinear'], 'penalty': ('l1', 'l2')},
    {'C': [1, 25], 'solver': ['saga'], 'penalty': ['elasticnet'], 'l1_ratio': [0.1, 1.0]} 
]

learn_save_score(log_regression, params, X_train, y_train, X_test, y_test, 'red', 'logreg')

Confusion matrix:
 [[ 16  27]
 [  9 268]]



,scores,logreg
0,Accuracy,0.887500
1,Recall,0.967509
2,Precision,0.908475
3,ROC AUC curve,0.669801


### Метод опорных векторов

In [15]:
from sklearn.svm import SVC

svm = SVC(max_iter=500)

params = [
    {'C': [1, 25], 'kernel': ['poly', 'rbf', 'sigmoid'], 'gamma': ['scale', 'auto']},
]

learn_save_score(log_regression, params, X_train, y_train, X_test, y_test, 'red', 'logreg')

ValueError: Invalid parameter 'gamma' for estimator LogisticRegression(C=1, max_iter=500). Valid parameters are: ['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'].

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier


    classifiers = {         #Словарь классификаторов и их параметров
        'logistic-regression': (LogisticRegression(solver='liblinear', max_iter=250, penalty='l2'),
                                    {'clf__C': uniform(loc=0.01, scale=2), 
                                    'clf__tol': uniform(loc=1e-8, scale=1e-2)}),
        'SVM':                  (SVC(kernel='linear', max_iter=250),
                                    {'clf__C': uniform(loc=0.01, scale=2),
                                    # 'clf__gamma': uniform(loc=0, scale=10), 
                                    # 'clf__degree': randint(2,5), 
                                    'clf__tol': uniform(loc=1e-8, scale=1e-2)}),
        'KNN':                  (KNeighborsClassifier(algorithm='kd_tree'),
                                    {'clf__n_neighbors': randint(3, 8), 
                                    'clf__leaf_size': randint(10, 60)}),
        'naive-bayes':          (GaussianNB(),
                                    {'clf__var_smoothing': uniform(loc=1e-10, scale=1)}),
        'random-forest':        (RandomForestClassifier(random_state=2022),
                                    {'clf__n_estimators': randint(100, 300), 
                                    'clf__max_features': ['sqrt', 'log2'],
                                    'clf__min_samples_split': randint(2, 5)})
    }

In [ ]:
learn_and_score('white', white_X_train, white_y_train, white_X_test, white_y_test, pipeline)

В итоге для обоих датасетов лучший результат показало применение случайного леса. С другой стороны, именно его обучение занимает больше всего времени. Для белого вина случайный лес показал лучшие результаты по всем параметрам, однако для красного метод SVM показал лучший Recall - что показывает, что модель в данной ситуации с большей вероятностью выберет правильно хорошие вина, но при этом в эту выборку могут попасть и плохие.